In [ ]:
!pip install -q pyomo
!apt-get install -y -qq glpk-utils

ابتدا کتاب خانه های مورد نیاز را ایمپورت می کنیم.

In [ ]:
import pyomo.environ as pyo
import numpy as np
import random
model = pyo.ConcreteModel()

In [ ]:
professors_number = 12
students_number = 30

c_i = np.array([1, 2 ,4, 4, 5, 2, 3, 2, 3, 2, 1, 1])

p_ij = []
for i in range(professors_number):
  p_i = list(k for k in range(1, students_number+1))
  random.shuffle(p_i)
  p_ij.append(p_i)
p_ij = np.array(p_ij)

q_ij = []
for i in range(students_number):
  q_i = list(k for k in range(1, professors_number+1))
  random.shuffle(q_i)
  q_ij.append(q_i)
q_ij = np.array(q_ij)

#Index
model.Iset = pyo.Set(initialize=range(1, professors_number+1))
model.Jset = pyo.Set(initialize=range(1, students_number+1))

#Params
def rule_p(model, i, j):
    return p_ij[i-1][j-1]
model.p = pyo.Param(model.Iset, model.Jset,initialize=rule_p)

def rule_q(model, i, j):
    return q_ij[j-1][i-1]
model.q = pyo.Param(model.Iset, model.Jset,initialize=rule_q)

def rule_c(model, i):
    return c_i[i-1]
model.c = pyo.Param(model.Iset, initialize=rule_c)


#Variables
model.delta = pyo.Var(model.Iset, model.Jset, domain=pyo.Binary)

#ObjectiveFunction
def rule_obj(model):
    return sum((model.p[i, j] + model.q[i, j])*model.delta[i, j] for i in model.Iset for j in model.Jset)
model.obj=pyo.Objective(rule=rule_obj, sense=pyo.minimize)


#Constraint
def rule_const1(model, j):
      return sum(model.delta[i, j] for i in model.Iset) == 1
model.const1=pyo.Constraint(model.Jset, rule=rule_const1)

def rule_const2(model, i):
      return sum(model.delta[i, j] for j in model.Jset) == model.c[i]
model.const2=pyo.Constraint(model.Iset, rule=rule_const2)


result=pyo.SolverFactory('glpk').solve(model, 'glpk')
print(result)




Problem: 
- Name: unknown
  Lower bound: 280.0
  Upper bound: 280.0
  Number of objectives: 1
  Number of constraints: 43
  Number of variables: 361
  Number of nonzeros: 721
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.014383077621459961
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [ ]:
print(p_ij)
print(q_ij)

[[24 25 30  9  8 13 27  6 19 26 10 29 28 16 20  7 14 15 22  3 17 23 18 11
   2  4 21  5 12  1]
 [13 29 28 25  6 23  1 11 16 19 27 18  2  9 30  7 12 24  8 15 14  5 26 17
  21  3 22 20  4 10]
 [24 30 25 14 10  5 20  6  2 17 18 28 13 15 21 12  8 27  7  1  9 19 22 29
  26  3 16 11 23  4]
 [22 11 27  2  6 21 25  9  5 18 20  7 24 14 23  1 17 13 15  8 19 29 10 30
   3  4 28 16 12 26]
 [ 7 18 10 26 29  6 15 27  2  3  9 23 16 30  8 11  1 21 20 14 28 17  5 22
  12 24 25  4 13 19]
 [23 30 11 21 13 19  8  9 17  6 14 15 18 16  5  2  4 10  3 29 25 27 20 22
  26 28 12  7 24  1]
 [11 18 15  2  5 24  6 12 10 23  1 17  7 14 19 28 29  3 20 16 27  9 13  4
  21 22  8 25 26 30]
 [26 30 10 20 14 28  3  7 18 16  8  4 27 29  6 12  2 24 21  5 25 13 19 22
  17  1 23 11  9 15]
 [ 7 28 14 27 21  9 10 22 12 17  4 15 29 24 18 26  6 13 11  1  8 25 16  2
   5  3 19 20 23 30]
 [ 9  6 20 17  4 23 10 19  1 14 27 25 16  8 21 12 29  3 18  7 30 26 11 28
   2 22 15 13 24  5]
 [26  7  3 10 14 18 25 29 12  8 15 27 21  1  9  4 

In [ ]:
print("Optimal value of delta")
students = []
professors = []
for i in model.Iset:
  prof_i = []
  for j in model.Jset:
    if  pyo.value(model.delta[i, j])==1:
        print(i, j, ": ", pyo.value(model.delta[i, j]))
        prof_i.append(j)
  professors.append(prof_i)

for j in model.Jset:
  for i in model.Iset:
    if  pyo.value(model.delta[i, j])==1:
        students.append(i)


Optimal value of delta
1 30 :  1.0
2 7 :  1.0
2 13 :  1.0
3 8 :  1.0
3 9 :  1.0
3 19 :  1.0
3 21 :  1.0
4 4 :  1.0
4 16 :  1.0
4 25 :  1.0
4 29 :  1.0
5 1 :  1.0
5 10 :  1.0
5 15 :  1.0
5 17 :  1.0
5 28 :  1.0
6 3 :  1.0
6 27 :  1.0
7 11 :  1.0
7 18 :  1.0
7 23 :  1.0
8 12 :  1.0
8 26 :  1.0
9 6 :  1.0
9 20 :  1.0
9 24 :  1.0
10 5 :  1.0
10 14 :  1.0
11 2 :  1.0
12 22 :  1.0


In [ ]:
professors

[[30],
 [7, 13],
 [8, 9, 19, 21],
 [4, 16, 25, 29],
 [1, 10, 15, 17, 28],
 [3, 27],
 [11, 18, 23],
 [12, 26],
 [6, 20, 24],
 [5, 14],
 [2],
 [22]]

In [ ]:
students

In [ ]:
for j in model.Jset:
  print("Student", j," is assined to professor", students[j -1], "which was her", model.q[students[j -1], j], "priority")

for i in model.Iset:
  for x in professors[i-1]:
    print("Professor", i," is assined to student", x, "which was her", model.p[i, x], "priority")

Student 1  is assined to professor 5 which was her 1 priority
Student 2  is assined to professor 11 which was her 5 priority
Student 3  is assined to professor 6 which was her 1 priority
Student 4  is assined to professor 4 which was her 6 priority
Student 5  is assined to professor 10 which was her 1 priority
Student 6  is assined to professor 9 which was her 3 priority
Student 7  is assined to professor 2 which was her 8 priority
Student 8  is assined to professor 3 which was her 9 priority
Student 9  is assined to professor 3 which was her 4 priority
Student 10  is assined to professor 5 which was her 6 priority
Student 11  is assined to professor 7 which was her 11 priority
Student 12  is assined to professor 8 which was her 3 priority
Student 13  is assined to professor 2 which was her 2 priority
Student 14  is assined to professor 10 which was her 1 priority
Student 15  is assined to professor 5 which was her 7 priority
Student 16  is assined to professor 4 which was her 9 priori